# RNN Sequence Prediction for Music Generation

Goal of this lecture is to train a network that is able to compose new music using a recurrent neural network (RNN) for sequence prediction.

---
### Course of Action

* Please write all executable python code in ```Code```-Cells (```Cell```->```Cell Type```->```Code```) and all Text as [Markdown](http://commonmark.org/help/) in ```Markdown```-Cells
* Describe your thinking and your decisions (where appropriate) in an extra Markdown Cell or via Python comments
* In general: discuss all your results and comment on them (are they good/bad/unexpected, could they be improved, how?, etc.). Furthermore, visualise your data (input and output).
* Write a short general conclusion at the end of the notebook
* Further experiments are encouraged. However, don't forget to comment on your reasoning.
* Use a scientific approach for all experiments (i.e. develop a hypothesis or concrete question, make observations, evaluate results)

### Submission

E-Mail your complete Notebook to [maucher@hdm-stuttgart.de](mailto:maucher@hdm-stuttgart.de) until the start of the next lecture. One Notebook per Group is enough. Edit the teammember table below.

**Important**: Also attach a HTML version of your notebook (```File```->```Download as```->```HTML```) in addition to the ```.ipynb```-File.

| Teammember |                    |
|------------|--------------------|
| 1.         | Christopher Caldwell |
| 2.         | Fabian Müller      |
| 3.         | An Dang         |

## Prerequisities
1. Make sure you have a general understanding how RNNs are able to take time-series input and predict the future. A general overview of RNNs and different architectures can be found in the Machine Learning lecture [here](http://nbviewer.jupyter.org/urls/maucher.home.hdm-stuttgart.de/nb/ML/RecurrentNeuralNetworks.ipynb). A practical example how future samples can be predicted can be found [here](http://nbviewer.jupyter.org/urls/maucher.home.hdm-stuttgart.de/nb/ML/LSTMtemperature.ipynb). 

2. Make sure to have the following python libraries installed (e.g. via ```pip install```)
    * tensorflow
    * keras (or use the [```tensorflow.keras```](https://www.tensorflow.org/api_docs/python/tf/keras) module
    * [music21](http://web.mit.edu/music21/)
    * scipy

## Understanding the Input

To be able to compose new music (that ideally does not sound like random notes), the network needs to *listen* to music. Intuitivley the network will try to create music that sounds similar (tone color, rythm, etc.) to the music it listened to during the training.

We use midi-files due to their wide availability to generate the input to our network. A good ressource of free, classical midi-files can be found on the [Classical Piano Midi Page](http://www.piano-midi.de), however you can use any other ressource of midi-files (However, it works best with files with a single *track* (a single instrument).

A midi-file is a simple series of commands (e.g. ```note_on``` or ```note_off```) with a given time offset. To parse the midi-files we use the [music21](http://web.mit.edu/music21/) library. This library offers a wide range of musical analysis functions. We, for now, use it to parse midi files and to normalize the input.

#### Tasks:
* Download a set of midi-files for training (e.g. all compositions of a single artist).
* Load one of the files using music21
* Print the parsed file-structure
* Show an inline player for the file in your browser

#### Hints:
* Use a [```music21.converter```](http://web.mit.edu/music21/doc/moduleReference/moduleConverter.html#module-music21.converter) to read the file
* Use the [```Music21Object.show()```](http://web.mit.edu/music21/doc/moduleReference/moduleBase.html#music21.base.Music21Object.show) method to render the file in different formats.

In [1]:
from music21 import converter
import os
sample_file = os.path.join('beeth', 'mond_1.mid')
sample_midi = converter.parse(sample_file)
sample_midi.show('midi')
sample_midi.show('text')

{0.0} <music21.stream.Part 0x1dba390cb00>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark Quarter=49.49>
    {0.0} <music21.key.Key of E major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x1dba3929b70>
        {0.0} <music21.note.Rest rest>
        {19.0} <music21.note.Note G#>
        {19.75} <music21.note.Note G#>
        {20.0} <music21.note.Note G#>
        {23.0} <music21.note.Note G#>
        {23.75} <music21.note.Note G#>
        {24.0} <music21.note.Note G#>
        {26.0} <music21.note.Note A>
        {28.0} <music21.note.Note G#>
        {30.0} <music21.note.Note F#>
        {31.0} <music21.note.Note B>
        {32.0} <music21.note.Note E>
        {39.0} <music21.note.Note G>
        {39.75} <music21.note.Note G>
        {40.0} <music21.note.Note G>
        {43.0} <music21.note.Note G>
        {43.75} <music21.note.Note G>
        {44.0} <music21.note.Note G>
        {47.0} <music21.note.Note F#>
        {48.0

As you can see, music21 automatically detects chords when multiple notes are played at the same time. This is helpful as this allows to represent multiple notes as a single symbol. Thus, the model can be trained on one-hot encoded data and still be able to generate polyphonic music.

The next step is to encode the songs into a list of tones. For simple music generation, we ignore the beat and tempo of the music and treat the songs as a series of equidistant notes or chords. 

#### Tasks:
* parse all midi files and put all notes and chords into a single list

#### Hints:
* Midi files may contain multiple *parts* (tracks, instruments, etc.) or are *flat* (notes and chords are not nested in parts). Use [```music21.stram.Stram.recurse()```](http://web.mit.edu/music21/doc/moduleReference/moduleStream.html?highlight=recurse#music21.stream.Stream.recurse) to walk over all files and flatten the structure.
* For [```music21.note.Note```](http://web.mit.edu/music21/doc/moduleReference/moduleNote.html#note) objects, add the ```pitch``` to the tones list. For [```music21.chord.Chord```](http://web.mit.edu/music21/doc/moduleReference/moduleChord.html#chord) objects, join all notes in the chord using a seperator (e.g. ```.```). It is important to narmalize the chord to its [```normalOrder```](http://web.mit.edu/music21/doc/moduleReference/moduleChord.html#music21.chord.Chord.normalOrder) to reduce the number of unique elements drastically without altering the traiing data too significantly. If the unnormalized chords are used, the traiing set is full of classes that only appear very rare.

In [ ]:
from music21 import converter, instrument, note, chord
import glob

notes = []
data_dirs = ('beeth', )

# load all files in all dirs
for path in data_dirs:
    for file in glob.glob(path + '/*.mid'):
        print('parsing file ' + file)
        midi = converter.parse(file)
        notes_to_parse = midi.recurse()
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

parsing file beeth\appass_1.mid
parsing file beeth\appass_2.mid
parsing file beeth\appass_3.mid
parsing file beeth\beethoven_hammerklavier_1.mid
parsing file beeth\beethoven_hammerklavier_2.mid
parsing file beeth\beethoven_hammerklavier_3.mid
parsing file beeth\beethoven_hammerklavier_4.mid
parsing file beeth\beethoven_les_adieux_1.mid
parsing file beeth\beethoven_les_adieux_2.mid
parsing file beeth\beethoven_les_adieux_3.mid
parsing file beeth\beethoven_opus10_1.mid
parsing file beeth\beethoven_opus10_2.mid
parsing file beeth\beethoven_opus10_3.mid


## Create training data

We train the network to predict the next note or chord after it has seen a sequence of previous tones. Therefore, each input element is a sequence of tones and the label is the next note or chord followed by this sequence.

Until now, each tone is represented by a string. For the output, this representation needs to be converted to one-hot encoded vectors to be able to train the model using categorical cross-entropy. The input needs to be mapped to floats in $[0,1|$ that are used as input to the network.

![preprocess](data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0D%0A%3C%21DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%0D%0A%3Csvg%20width%3D%22100%25%22%20height%3D%22100%25%22%20viewBox%3D%220%200%202953%20591%22%20version%3D%221.1%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20xml%3Aspace%3D%22preserve%22%20xmlns%3Aserif%3D%22http%3A%2F%2Fwww.serif.com%2F%22%20style%3D%22fill-rule%3Aevenodd%3Bclip-rule%3Aevenodd%3Bstroke-linecap%3Around%3Bstroke-linejoin%3Around%3Bstroke-miterlimit%3A1.5%3B%22%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C965.827%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EF4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1068.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1045.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3ED4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1150.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1130.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EF4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1232.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1209.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1314.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1292.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EE4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1396.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1374.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1478.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1456.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1560.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1531.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1232.83%2C96%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1209.83%2C118%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1314.83%2C96%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1292.83%2C118%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EE4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1396.83%2C96%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1374.83%2C118%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1478.83%2C96%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1456.83%2C118%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1560.83%2C96%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28166%2C219%2C223%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1531.83%2C118%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1642.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1609.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EF%234%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1314.83%2C236%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1292.83%2C258%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EE4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1396.83%2C236%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1374.83%2C258%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1478.83%2C236%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1456.83%2C258%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1560.83%2C236%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1531.83%2C258%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1642.83%2C236%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28166%2C219%2C223%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1609.83%2C258%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EF%234%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1724.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1701.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC%23%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1396.83%2C376%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1374.83%2C398%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1478.83%2C376%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1456.83%2C398%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1560.83%2C376%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1531.83%2C398%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1642.83%2C376%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1609.83%2C398%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EF%234%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1724.83%2C376%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28166%2C219%2C223%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1701.83%2C398%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC%23%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1806.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1784.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1888.83%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28202%2C203%2C201%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1866.83%2C-22%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB5%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M331.35%2C186.521L331.36%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M659.35%2C186.521L659.35%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M413.35%2C326.521L413.35%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M741.35%2C326.521L741.35%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M495.36%2C466.414L495.35%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M823.36%2C466.414L823.36%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C985.827%2C-44%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M905.36%2C466.521L905.36%2C128.531%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A1px%3Bstroke-dasharray%3A3%2C3%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C1601.35%2C-303.498%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C1683.35%2C-163.498%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C1765.35%2C-23.4975%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1103.37%2C6.19147%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3EX%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C437.584%2C-364.362%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3Eseq%3C%2Ftext%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22602.974px%22%20y%3D%22590.044px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A36.438px%3B%22%3E4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C383.584%2C-514.362%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3Enotes%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C437.584%2C-234.362%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3Eseq%3C%2Ftext%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22602.974px%22%20y%3D%22590.044px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A36.438px%3B%22%3E5%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C437.584%2C-94.3615%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3Eseq%3C%2Ftext%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22602.974px%22%20y%3D%22590.044px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A36.438px%3B%22%3E6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%28-0.00034298%2C1%2C-1%2C-0.00034298%2C1573.35%2C-16.572%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3E...%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%28-0.00034298%2C1%2C-1%2C-0.00034298%2C1570.03%2C-421.634%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3E...%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1314.37%2C6.19147%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3Ey%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.00023754%2C0.71323%2C-4.12755%2C0.00137467%2C4031.46%2C247.776%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22353.101px%22%20y%3D%22598.079px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A79.542px%3B%22%3E%7D%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.00023754%2C0.71323%2C-1.03072%2C0.000343278%2C2446.05%2C248.332%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22353.101px%22%20y%3D%22598.079px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A79.542px%3B%22%3E%7D%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%3C%2Fsvg%3E%0D%0A)

#### Tasks: 
* Create the training elements by following the *shifting* procedure depicted above
* normalize the input sequences to $[0,1|$ and one-hot encode the labels

#### Hints:
* Use a sequence length of ~50-100 notes
* create a mapping from the string-representation of a tone to an int and vice versa
* use [```keras.utils.to_categorical```](https://keras.io/utils/#to_categorical) to convert an integer to a one-hot vector

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

sequence_length = 100
# get all pitch names
pitchnames = sorted(frozenset(notes))
# create a dictionary to map pitches to integers
note_to_int = {note: number for number, note in enumerate(pitchnames)}
int_to_note = {number: note for number, note in enumerate(pitchnames)}

X = []
y = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    X.append([note_to_int[char] for char in sequence_in])
    y.append(note_to_int[sequence_out])

# reshape the input into a format compatible with LSTM layers
X = np.reshape(X, (-1, sequence_length, 1))

# normalize input and output
X = X / len(pitchnames)
y = to_categorical(y)

print(X.shape)
print(y.shape)

## Create and train the Network

With the prepared training data, it is now possible to train the network. There are many variants of RNNs (e.g. unidirectional vs. bidirectional and LSTM vs. GRU). You can encouraged to try your own network architecture, however here is an example architecture as a starting point:

* 3 consecutive unidirectional recurrent layer with ```512``` [LSTM](https://keras.io/layers/recurrent/#lstm) units, returning all sequence states (not only the last state) to feed into the next recurrent layer (except for the last layer) and [Dropout](https://keras.io/layers/core/#dropout) with a rate of ```0.3``` between each layer
* a [Dense](https://keras.io/layers/core/#dense) layer with ```256``` neurons, ReLU activation and a dropout rate of ```0.3```
* another Dense layer for the output with softmax activation
* RMSProp optimizer using categorical-crossentropy as lossfunction

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential()
model.add(LSTM(
    512,
    return_sequences=True, 
    input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(rate=0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(512))
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(len(pitchnames), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

As the training of a complex RNN can take a very long time, you should make sure to add a [ModelCheckpoint](https://keras.io/callbacks/#modelcheckpoint) callback to your model to regularly save the parameters during training.

#### Tasks:
* train your model for 50-100 epochs

#### Hints:
* train your model on a reduced training set and for a single epoch first to make sure you have no programming errors
* the training can take a very long time, use the [Deeplearn Machines @ HDM](https://wiki.mi.hdm-stuttgart.de/wiki/Deep_Learning) to speed up training (significantly!)

In [ ]:
checkpoint = ModelCheckpoint(
    'newchpts',
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

callbacks_list = [checkpoint]
model.fit(X, y, epochs=100, batch_size=1024, callbacks=callbacks_list)

# Generating new Music

After training, the model should have learned to predict the note or chord after *listening* to a sequence of tones. If the model is good at this task, it means that it does know what sequence of tones *sounds right™* (given the training data has the same property).

To generate new music, you just need to let the network *listen* to a sequence of tones (the seed) and ask for the next note or chord. Then, append this predicted tone to the end of the seed (and remove the first entry), feed the new seed into the network and get for the next tone.

![rnn generation](data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0D%0A%3C%21DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%0D%0A%3Csvg%20width%3D%22100%25%22%20height%3D%22100%25%22%20viewBox%3D%220%200%202953%20591%22%20version%3D%221.1%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20xml%3Aspace%3D%22preserve%22%20xmlns%3Aserif%3D%22http%3A%2F%2Fwww.serif.com%2F%22%20style%3D%22fill-rule%3Aevenodd%3Bclip-rule%3Aevenodd%3Bstroke-linecap%3Around%3Bstroke-linejoin%3Around%3Bstroke-miterlimit%3A1.5%3B%22%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C506.827%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C483.827%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C588.827%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C566.827%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EE4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C670.827%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C648.827%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C752.827%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C730.827%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C588.827%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C566.827%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EE4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C670.827%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C648.827%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C752.827%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C730.827%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C834.827%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C805.827%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C934.827%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C905.827%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1576.83%2C26%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1547.83%2C48%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1576.83%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1547.83%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C670.827%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C648.827%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EC6%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C752.827%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C730.827%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C834.827%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C805.827%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C916.827%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C893.827%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1017.83%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C994.827%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1659.83%2C166%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1636.83%2C188%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1576.83%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1547.83%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1659.83%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1636.83%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C752.827%2C446%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28208%2C124%2C127%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C730.827%2C468%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C834.827%2C446%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C805.827%2C468%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C916.827%2C446%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C893.827%2C468%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C998.827%2C446%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C978.827%2C468%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EA4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1095.83%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1076.83%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EA4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1742.83%2C306%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1723.83%2C328%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EA4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1576.83%2C436%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1547.83%2C458%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EB-4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1659.83%2C436%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1636.83%2C458%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EG4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1742.83%2C436%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1824.83%2C436%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Crect%20x%3D%2284.35%22%20y%3D%2246.521%22%20width%3D%2282.01%22%20height%3D%2282.01%22%20style%3D%22fill%3Argb%28151%2C208%2C119%29%3Bstroke%3Ablack%3Bstroke-width%3A1px%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1723.83%2C458%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22115.625px%22%20y%3D%2283.587px%22%20style%3D%22font-family%3A%27ArialMT%27%2C%20%27Arial%27%2C%20sans-serif%3Bfont-size%3A50px%3B%22%3EA4%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C796.352%2C-373.498%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C878.844%2C-233.498%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C957.352%2C-93.4975%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%280.50007%2C0%2C0%2C1%2C1039.35%2C46.5025%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M275.736%2C494.2L275.736%2C479.848L374.611%2C479.848L374.611%2C466.521L415.61%2C487.024L374.611%2C507.526L374.611%2C494.2L275.736%2C494.2Z%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C205.216%2C-531.178%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A37.5px%3B%22%3Eseed%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C460.216%2C-531.178%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A37.5px%3B%22%3Eprediction%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C745.116%2C-32.5778%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3E...%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1406.52%2C-42.2273%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A62.5px%3B%22%3E...%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%282.01933e-16%2C-0.71323%2C4.12755%2C1.16899e-15%2C-1631.49%2C328.808%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22353.101px%22%20y%3D%22598.079px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A79.542px%3B%22%3E%7D%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%282.01933e-16%2C-0.71323%2C4.12755%2C1.16899e-15%2C-561.494%2C328.808%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22353.101px%22%20y%3D%22598.079px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A79.542px%3B%22%3E%7D%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%282.01933e-16%2C-0.71323%2C1.04117%2C2.94875e-16%2C458.116%2C328.808%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22353.101px%22%20y%3D%22598.079px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A79.542px%3B%22%3E%7D%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%281%2C0%2C0%2C1%2C1123.22%2C-531.178%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Ctext%20x%3D%22517.599px%22%20y%3D%22569.231px%22%20style%3D%22font-family%3A%27LMMath-Regular%27%2C%20%27Latin%20Modern%20Math%27%3Bfont-size%3A37.5px%3B%22%3Egenerated%20comp%3Ctspan%20x%3D%22777.062px%20%22%20y%3D%22569.231px%20%22%3Eo%3C%2Ftspan%3Esition%3C%2Ftext%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%20%20%20%20%3Cg%20transform%3D%22matrix%284.14714e-16%2C-6.7728%2C6.7728%2C4.14714e-16%2C1280.62%2C11356.2%29%22%3E%0D%0A%20%20%20%20%20%20%20%20%3Cpath%20d%3D%22M1672.64%2C28.679L1593.64%2C28.904%22%20style%3D%22fill%3Anone%3Bstroke%3Ablack%3Bstroke-width%3A0.15px%3Bstroke-dasharray%3A1.48%2C1.48%2C0%2C0%3B%22%2F%3E%0D%0A%20%20%20%20%3C%2Fg%3E%0D%0A%3C%2Fsvg%3E%0D%0A)

#### Tasks:
* Implement the procedure described above to generate a sequence of 500 notes or chords.

#### Hints:
* Use a sample from the training data as the seed. This does not mean that the network will generate the same melody as in the song the seed is chosen from. As we use the prediction in the next round to predict the next tone, imperfections in the network's predictions will propagate and thus the generated output will *drift* from the seed more and more.
* Do not include the seed in your generated song (that would be cheating, right?)

In [ ]:
from tqdm import tnrange

start = np.random.randint(0, len(X)-1)

pattern = X[start]
prediction_output = []
# generate 500 notes
for note_index in tnrange(500, desc='generating notes'):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(len(pitchnames))
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    
    pattern = np.vstack((pattern, index))
    pattern = np.delete(pattern, 0, 0)

Now that you have a list of tones in integer representation, you need to convert it back to ```music21.note.Note``` and ```music21.chord.Chord``` objects that are arranged in a flat midi-stream.

#### Tasks:
* convert the generated song to a ```music21.stream.Stream``` by first mapping each int back to the string representation and then creating notes and chords.

#### Hints:
* for notes you can simply pass the string representation to the ```music21.note.Note``` constructor 
* for chords you first need to split the string into single notes and assemble them into a ```music21.chord.Chord```
* as our model is not able to predict any property other than the next note/chord, we can only use a fixed tone duration. Give each chord/note a fixed offet of $\frac{1}{2}$ bar.
* specify a [```music21.Instrument()```](http://web.mit.edu/music21/doc/moduleReference/moduleInstrument.html#piano) to each note

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    if '.' in pattern:
        # pattern is a chord
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        # pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

You can now use the ```show()``` method of the stream to listen to the generated masterpiece or save it to disk for later enjoyment using the ```write()``` method.

In [ ]:
from music21 import stream

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')
midi_stream.show('midi')

## Further Ideas

### Simple:
* Experiment with different network architectures (GRU, Bidirectional, ...) (As training always takes a long time, this may produce limited outcome)
* Experiment with the effect of different hyperparameters. For example a very short sequence length speeds up training quite a bit, how does the generated output change (which patterns do occur)

### Advanced:
* Use a different loss function (maybe use the 'closeness' to the correct note instead of cross-entropy as the notes are not completely seperate classes, maybe employ some music theory if you have any)
* Use more input dimensions. Maybe add the duration of a tone or other properties.
* Use distributed representations of tones. In a distributed embedding space, similarities between notes could be mapped. You could learn the embedding using the skip-gram method on the note-entities.
* Use a completley different method to generate music. For example [waveGANs](https://github.com/chrisdonahue/wavegan/blob/master/README.md) generate sound using raw ausio samples. You could render your midi into wav and use the waveGAN technique or encode your data in another way and try to create your own GAN architecture.
* try to let the network predict different tone lengths (you could quantize length to e.g. 3 different lengths ($\frac{1}{4}$ bar resolution) and encode the combination of tone+duration as a single entity. As this would increase the input and output space by a factor of 3, a distributed representation of tones could help)

#### Disclaimer:
All ideas above are untested, so I can't promise a successful outcome. Of course you can always implement your own ideas.